Load model and embedding data.

In [11]:
import theano
import theano.tensor as T
import numpy as np
import cPickle
import random
from utils import *
from state import *
from discri_model import DiscriModel

theano.config.floatX='float32'

model_name = 'model/train_discri_full_emb100_h256_valid20_model.npz'

state = prototype_state()
model = DiscriModel(state)
model.load(model_name)

print('Model loaded.')

(word2ind, ind2word) = cPickle.load(open('tmp/dict.pkl'))
acttype = cPickle.load(open('tmp/acttype_cnt.pkl'))
print acttype.keys()

print('Data loaded.')

Model loaded.
[u'requestaddr', u'informpricerange', u'requestarea', u'negate', u'requestpricerange', u'reqmore', u'informname', u'denyfood', u'confirmpricerange', u'requestsignature', u'requestphone', u'informarea', u'reqalts', u'informfood', u'denyname', u'repeat', u'confirmarea', u'requestfood', u'informthis', u'requestname', u'restart', u'affirm', u'bye', u'thankyou', u'ack', u'confirmfood', u'hello', u'requestpostcode']
Data loaded.


Now we try to mannually provide an input sentence (if a word is out of vocab, we simply remove it).

In [18]:
test_sent = 'moderately priced swedish food'
words = test_sent.split()
nat_coded = [1]
for w in words:
    if w in word2ind:
        nat_coded.append(word2ind[w])
    else:
        print '  out of vocab: %s' % w
nat_coded.append(0)
print 'Coded input:', nat_coded

m = state['seqlen']
nat_coded_mat = numpy.zeros((m, 1), dtype='int32')
sent_len = len(nat_coded)
nat_coded_mat[:sent_len, 0] = nat_coded
pred_fn = theano.function([model.natural_input],
                                model.pred)
pred = pred_fn(nat_coded_mat)

print 'Prediction (coded):', pred

def decode_act(acttype, pred):
    res = {}
    pos = 0
    for act in acttype.keys():
        res[act] = []
        k = max(1, acttype[act])
        print k
        while k > 0:
            res[act].append(pred[pos])
            pos = pos + 1
            k = k - 1
    return res

res = decode_act(acttype, pred)
for act in res.keys():
    print "%s: %s" % (act, ' '.join([ind2word[ind] for ind in res[act]]))

Coded input: [1, 273, 661, 144, 467, 0]
Prediction (coded): [539 539 539 539 539 539 539   4   4   4 539 539 539 539 539 539 539   4
 539   4   4 539 539 539 539 539 539 539 539 539 539 539   4 539 539]
1
1
1
1
1
1
4
1
1
1
1
1
1
2
3
1
1
1
1
1
1
1
1
1
1
2
1
1
requestaddr: <NO>
informpricerange: <NO>
thankyou: <NO>
requestarea: <NO>
negate: <NO>
requestpricerange: <NO>
reqmore: <NO>
informname: <NO> <NULL> <NULL> <NULL>
denyfood: <NO>
confirmpricerange: <NO>
requestsignature: <NO>
requestphone: <NO>
informarea: <NO>
reqalts: <NO>
informfood: <NO> <NULL>
denyname: <NO> <NULL> <NULL>
repeat: <NO>
confirmarea: <NO>
requestfood: <NO>
informthis: <NO>
requestname: <NO>
restart: <NO>
bye: <NO>
affirm: <NO>
ack: <NO>
confirmfood: <NO> <NULL>
hello: <NO>
requestpostcode: <NO>
